# Data Skipping Sample for Python

[Data skipping](https://cloud.ibm.com/docs/services/AnalyticsEngine?topic=AnalyticsEngine-data-skipping&locale=en) can significantly boost the performance of SQL queries by skipping over irrelevant data objects or files based on a summary metadata associated with each object.

For every column in the object, the summary metadata might include minimum and maximum values, a list or bloom filter of the appearing values, or other metadata which succinctly represents the data in that column. This metadata is used during query evaluation to skip over objects which have no relevant data.

All Spark native data formats are supported, including Parquet, ORC, CSV, JSON and Avro. Data skipping is a performance optimization feature which means that using data skipping does not affect the content of the query results.

To use this feature, you need to create indexes on one or more columns of the data set. After this is done, Spark SQL queries can benefit from data skipping. In general, you should index the columns which are queried most often in the WHERE clause.

### Table of Contents:
* [1. Setup the environment](#cell0)
* [2. Creating a sample dataset](#cell1)
* [3. Setup the DataSkipping library](#cell2)
    * [3.1 Indexing a dataset](#cell2.1)
* [4. Using the data skipping indexes ](#cell3)
    * [4.1 Running queries](#cell3.2)
* [Authors](#authors)

<a id="cell0"></a>
## Setup the environment

In [1]:
from metaindex import MetaIndexManager

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200506210714-0000
KERNEL_ID = b6ee6de9-9b19-4176-b068-76c352a33a2b


**Optional:** set log level to DEBUG for the metaindex package to view the skipped objects.

In [2]:
log4jLogger = spark.sparkContext._jvm.org.apache.log4j
log4jLogger.LogManager.getLogger('com.ibm.metaindex.search').setLevel(log4jLogger.Level.DEBUG)

### Configure Stocator
For more info on how to config credentials see [here](https://github.com/CODAIT/stocator)

See [here](https://cloud.ibm.com/docs/services/cloud-object-storage?topic=cloud-object-storage-endpoints) for the list of endpoints.
If you are in Cloud, make sure you choose the private endpoint of your bucket.

In [3]:
hconf = spark.sparkContext._jsc.hadoopConfiguration()
hconf.set("fs.cos.service.endpoint" ,"https://s3.private.us-south.cloud-object-storage.appdomain.cloud")
hconf.set("fs.cos.service.access.key", "")
hconf.set("fs.cos.service.secret.key","")

<a id="cell1"></a>
## Creating a sample dataset

Create a sample dataset consisting of 2 rows each row in a different object to demonstrate data skipping.
Please replace `dataset_location` with a location to save the sample dataset.

In [4]:
from pyspark.sql.types import *

dataset_location = "cos://<mybucket>.service/tmp/sampleskipping" # i.e. dataset_location = cos://guytestssouth.service/tmp/sampleskipping 

dfSchema = StructType([StructField("dt", StringType(), True), StructField("temp", DoubleType(), True),\
                      StructField("city", StringType(), True), StructField("vid", StringType(), True)]) 

data = [("2017-07-07", 20.0, "Tel-Aviv", "a"), ("2017-07-08", 30.0, "Jerusalem", "b")]

df = spark.createDataFrame(data, schema=dfSchema) 

# use partitionBy to make sure we have two objects
df.write.partitionBy("dt").mode("overwrite").parquet(dataset_location)

<a id="cell2"></a>
## Setup the DataSkipping library
In this example, we will set the JVM wide parameter to a base path to store all of the indexes. 

Metadata can be stored on the same storage system as the data however, not under the same path. For more configuration options, see [Data skipping configuration options](https://cloud.ibm.com/docs/services/AnalyticsEngine?topic=AnalyticsEngine-data-skipping-config-options&locale=en).
Please replace `md_base_location` with a location to save the sample dataset

In [5]:
md_base_location = "cos://<mybucket>.service/tmp/sampleskippingmetadata" # e.g. md_base_location = cos://guytestssouth.service/tmp/sampleskipping 
MetaIndexManager.setDefaultMetaDataStore(spark, 'com.ibm.metaindex.metadata.metadatastore.parquet.Parquet')
md_backend_config = dict([('spark.ibm.metaindex.parquet.mdlocation', md_base_location),
("spark.ibm.metaindex.parquet.mdlocation.type", "EXPLICIT_BASE_PATH_LOCATION")])
MetaIndexManager.setConf(spark, md_backend_config)

<a id="cell2.1"></a>
### Indexing a dataset

Skip this step if the data set is already indexed.

When creating a data skipping index on a data set, first decide which columns to index, then choose an index type for each column.
These choices are workload and data dependent. Typically, choose columns to which predicates are applied in many queries.
Currently the following index types are supported:
1. Min/max – stores the minimum and maximum values for a column. Applies to all types except complex types.
2. Value list – stores the list of values appearing in a column. Applies to all types except complex types.
3. Bloom Filter – stores bloom filter with false positive probability of 1%. Applies to ByteType, StringType, LongType, IntegerType, and ShortType.


- Choose value list if the number of distinct values in an object is typically much smaller than the total number of values in that object
- Bloom filters are recommended for columns with high cardinality.
- (otherwise the index can get as big as that column in the data set).

In [6]:
md_backend = 'com.ibm.metaindex.metadata.metadatastore.parquet.ParquetMetadataBackend'
reader = spark.read.format("parquet")
im = MetaIndexManager(spark, dataset_location, md_backend)

# remove existing index first
if im.isIndexed():
	im.removeIndex()
    
# indexing
im.indexBuilder()\
  .addMinMaxIndex("temp")\
  .addValueListIndex("city")\
  .addBloomFilterIndex("vid")\
  .build(reader)

print("finished indexing")

finished indexing


Note that each of the index types has a corresponding method in the indexBuilder class of the form:

`add[IndexType]Index(<index_params>)`

For example:

`addMinMaxIndex(col: String)`

`addValueListIndex(col: String)`

`addBloomFilterIndex(col: String)`

**(optional)** to refresh an indexed dataset use

In [7]:
im.refreshIndex(reader).show(10, False)

+-------+-----------------+-------------------+
|status |new_entries_added|old_entries_removed|
+-------+-----------------+-------------------+
|SUCCESS|0                |0                  |
+-------+-----------------+-------------------+



view index status

In [8]:
im.indexStats(reader).show(10, False)

+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+
|Data Skipping Index Stats|cloud-object-storage-my-cos-standard-9f6/tmp/sampleskipping                                                                                       |
+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+
|Status                   |Up to date                                                                                                                                        |
|Total objects indexed    |2                                                                                                                                                 |
|# Metadata properties    |                                                                                                  

<a id="cell3"></a>
## Using the data skipping indexes 

### Injecting the data skipping rule and enabling data skipping
the rule injection should be done only once per Spark session.

In [9]:
# inject the data skipping rule
MetaIndexManager.injectDataSkippingRule(spark)

# enable data skipping
MetaIndexManager.enableFiltering(spark)

# you can disable the data skipping any time by running: 
# MetaIndexManager.disableFiltering(spark)`

<a id="cell3.2"></a>
### Running queries

#### Load the dataframe

In [10]:
df = reader.load(dataset_location)
df.createOrReplaceTempView("metergen")

Example query which uses the min/max index:

In [11]:
spark.sql("select count(*) from metergen where temp < 30").show()

+--------+
|count(1)|
+--------+
|       1|
+--------+



View the data skipping statistics as follows.

In [12]:
MetaIndexManager.getLatestQueryAggregatedStats(spark).show(10, False)

+-------+-----------+-------------+------------+-----------+----------+
|status |isSkippable|skipped_Bytes|skipped_Objs|total_Bytes|total_Objs|
+-------+-----------+-------------+------------+-----------+----------+
|SUCCESS|true       |869          |1           |1729       |2         |
+-------+-----------+-------------+------------+-----------+----------+



**Optional:** clear the stats for the next query (otherwise, stats will acummulate)


In [13]:
MetaIndexManager.clearStats(spark)

Example query which uses value list index:

In [14]:
spark.sql("select count(*) from metergen where city IN ('Jerusalem', 'Ramat-Gan')").show()

+--------+
|count(1)|
+--------+
|       1|
+--------+



View the data skipping statistics:

In [15]:
MetaIndexManager.getLatestQueryAggregatedStats(spark).show()

+-------+-----------+-------------+------------+-----------+----------+
| status|isSkippable|skipped_Bytes|skipped_Objs|total_Bytes|total_Objs|
+-------+-----------+-------------+------------+-----------+----------+
|SUCCESS|       true|          860|           1|       1729|         2|
+-------+-----------+-------------+------------+-----------+----------+



**Optional:** clear the stats for the next query (otherwise, stats will acummulate)

In [16]:
MetaIndexManager.clearStats(spark)

Example query which uses bloom filter index.

In [17]:
spark.sql("select count(*) from metergen where vid = 'abc'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



### View the data skipping statistics

In [18]:
MetaIndexManager.getLatestQueryAggregatedStats(spark).show()

+-------+-----------+-------------+------------+-----------+----------+
| status|isSkippable|skipped_Bytes|skipped_Objs|total_Bytes|total_Objs|
+-------+-----------+-------------+------------+-----------+----------+
|SUCCESS|       true|         1729|           2|       1729|         2|
+-------+-----------+-------------+------------+-----------+----------+



**Optional:** clear the stats for the next query (otherwise, stats will acummulate)

In [19]:
MetaIndexManager.clearStats(spark)

<a id="authors"></a> 
### Authors

**Guy Khazma**, Cloud Data Researcher of IBM.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>